In [1]:
import import_ipynb
from base.datasets import load_real_path, load_real_data, real_data_convert
from base.train_model_birds import train_model, test_model

import os
import time
import pandas as pd
import random
import torch

importing Jupyter notebook from /scratch/Eugene/codes/grt_hist/StatEcoNet/StatEcoNet/base/datasets.ipynb
importing Jupyter notebook from /scratch/Eugene/codes/grt_hist/StatEcoNet/StatEcoNet/base/train_model_birds.ipynb
importing Jupyter notebook from /scratch/Eugene/codes/grt_hist/StatEcoNet/StatEcoNet/base/models.ipynb


# Define experiment setup

In [2]:
# Define the dataset
species_list = [
                #"Eurasian Collared-Dove",
                #"Common Yellowthroat",
                "Pacific Wren",
                #"Song Sparrow",
                #"Western Meadowlark"
                ]
fold_list = [1] # 1, 2, or 3

# Define the model
model_list = [2] # 0:OD-LR, 1:OD-1NN, 2:StatEcoNet

In [3]:
# Other parameters
max_iter = 1000
repeat = 1
save_flag = True

In [4]:
# Optimal hyper-parameters
opts = pd.read_csv("opt/opt_bird_NN.csv")

# Set the save path
if not os.path.isdir("output/"):
    os.mkdir("output/")
if not os.path.isdir("output/prediction/"):
    os.mkdir("output/prediction/")
save_base_path = "output/prediction/birds/"
if not os.path.isdir(save_base_path):
    os.mkdir(save_base_path)
test_file = "output/NN_bird_test.csv"

# Train/evalute the model

In [5]:
NN_record = pd.DataFrame(columns=['species','fold','nSites','nVisits',\
                                  'model.id','trial','best.iter',\
                                  'opt.learning.time','test.auroc',\
                                  'test.auprc'])

i = 0
start_init = time.time()
for species_name in species_list:
    dir_path = load_real_path(species_name)
    for fold in fold_list:
        # Load data
        x_dim, w_dim, k,\
        train_occCovars, train_detCovars, train_Y,\
        valid_occCovars, valid_detCovars, valid_Y,\
        test_occCovars, test_detCovars, test_Y = \
        load_real_data(dir_path, fold)

        x_train, w_train, y_train, x_valid, w_valid, y_valid,\
        x_test, w_test, y_test = \
        real_data_convert(train_occCovars, train_detCovars, train_Y,\
                          valid_occCovars, valid_detCovars, valid_Y,\
                          test_occCovars, test_detCovars, test_Y)

        nSites = x_train.shape[0]
        data_size = str(nSites)+"x"+str(k)

        for model_id in model_list:
            print("species", species_name, "test fold", fold, \
                  "model", model_id)
            opt = opts[(opts.speices==species_name) & (opts.fold==fold) & \
                       (opts.model_id==model_id)]

            lr = opt['lr'].item()
            bs = opt['batch_size'].item()
            if bs==0: bs = nSites
            else: bs = int(bs)
            nN = int(opt['nNeurons'].item())
            nL = int(opt['nLayers'].item())
            mixed = opt['mixed_weight'].item()

            for trial in list(range(1, repeat+1)):
                my_seed = random.randint(1, 1000)
                print("(", trial, "/", repeat, ")")

                # Find the optimal iteration    
                df_train, df_valid, \
                best_iter, best_model, opt_learing_time = \
                train_model(my_seed, model_id, nL, nN, lr, "train", \
                            x_dim, w_dim, k, max_iter, mixed, \
                            x_train, w_train, y_train, x_valid, \
                            w_valid, y_valid, bs)
                df_valid['dataset'] = 'valid'

                # Test the trained model
                psi_hat, p_hat, y_hat, auroc, auprc = \
                                test_model(best_model, \
                                           x_test, w_test, y_test, k)

                # Save prediction outcomes
                if save_flag:                               
                    save_path = save_base_path + species_name + \
                                "_f" + str(fold) + "_m" + str(model_id) \
                                + "_w" + str(mixed)  + "_t" + str(trial)
                    print("The model predictions are saved at", save_path)

                    pd.DataFrame(psi_hat.numpy().flatten()).to_csv( \
                                 save_path+'_psi_hat.csv', header=False, index=False)
                    pd.DataFrame(p_hat.numpy().flatten()).to_csv( \
                                 save_path+'_p_hat.csv', header=False, index=False)
                    pd.DataFrame(y_hat).to_csv( \
                                 save_path+'_y_hat.csv', header=False, index=False)                      
                    torch.save(best_model.state_dict(), \
                               save_path + '_final_model.pt')

                    record = [species_name, fold, nSites, k, model_id,\
                              trial, best_iter, opt_learing_time]
                    record.extend([auroc, auprc])
                    NN_record.loc[i] = record
                    NN_record.to_csv(test_file, index=False)

                i = i + 1

print("Test evaluation metrics are saved at", test_file)
end = time.time()
elapse = end - start_init
print(elapse)

data path: ../data/OR2020/Pacific Wren/
species Pacific Wren test fold 1 model 2
( 1 / 1 )


No more improvement. This is the early stop point.

The model predictions are saved at output/prediction/birds/Pacific Wren_f1_m2_w0.01_t1
Test evaluation metrics are saved at output/NN_bird_test.csv
28.312368869781494
